Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [2]:
import lab9_lib
from copy import deepcopy
from random import random, choice,choices, randint
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from copy import copy
from dataclasses import dataclass
import numpy as np
from pprint import pprint
import math

from importlib import reload
reload(lab9_lib)

<module 'lab9_lib' from 'c:\\Users\\iplov\\Desktop\\CI 2023\\My repo\\Computational-Intelligence-23-24\\Lab9\\lab9_lib.py'>

In [3]:
POPULATION_SIZE1 = 30  #
OFFSPRING_SIZE1 = 20  # ==> steady state algorithm

POPULATION_SIZE2 = 30 
OFFSPRING_SIZE2 = 20
NUM_ELITISM = POPULATION_SIZE2 - OFFSPRING_SIZE2   # Generational + elitism


TOURNAMENT_SIZE = 4
MUTATION_PROBABILITY = .15
NUM_LOCI = 1000

@dataclass
class Individual:
    fitness: int
    genotype: list[int]

In [6]:
def select_parent(pop): #tournament selection
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = copy(ind)
    pos = randint(0,NUM_LOCI - 1)
    offspring.genotype[pos] = 1 - offspring.genotype[pos]
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual: #cambia
    cut_point = randint(0,NUM_LOCI - 1)
    offspring = Individual(fitness=None,
                           genotype = ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == NUM_LOCI
    return offspring

def n_cut_xover(ind1: Individual, ind2: Individual, n:int) -> Individual: #cambia
    
    # Genera casualmente i punti di taglio
    cut_points = sorted([randint(0,NUM_LOCI - 1) for _ in range(n-1)])
    # Inizializza i vettori dei figli
    offspring = copy(ind1)

    # Alterna i segmenti tra i genitori
    for i in range(0,n-1,2):
        start = 0 if i == 0 else cut_points[i-1]
        end = cut_points[i]
        offspring.genotype[start:end] = ind2.genotype[start:end]

    # Completa gli ultimi segmenti
    if n%2!=0:
        offspring[cut_points[-1]:] = ind2[cut_points[-1]:]

    return offspring

def uniform_xover(ind1: Individual, ind2: Individual) -> Individual:
    offspring_genotype = []
    for i in range(NUM_LOCI):
        if random() > 0.5:
            offspring_genotype.append(ind2.genotype[i])
        else:
            offspring_genotype.append(ind1.genotype[i])

    offspring = Individual(genotype = offspring_genotype, fitness=None)  # Assicurati di avere un costruttore appropriato in Individual
    return offspring


# Problem instances 1 on a 1000-loci

In [9]:
n_instances = 1

Generational + Elitism

In [11]:
fitness_avg = 0
calls_avg = 0
trial = 10
for i in range(trial):
    population = [
        Individual(genotype=[choice((1, 0)) for _ in range(NUM_LOCI)], fitness=None)
        for _ in range(POPULATION_SIZE2)
    ]

    fitness = lab9_lib.make_problem(n_instances)
    for i in population:
        i.fitness = fitness(i.genotype)
    population.sort(key=lambda i: i.fitness, reverse=True)
    pprint(population[0].fitness)
    best = population[0].fitness
    count = 0
    for generation in range(10000):
        offspring = list()
        for o in range(OFFSPRING_SIZE2):
            if random() < MUTATION_PROBABILITY: # self-adapt mutation probability
                # do mutation  # add more clever mutations
                p = select_parent(population)
                o = mutate(p)
            else:
                # crossover     # add more xovers
                p1 = select_parent(population)
                p2 = select_parent(population)
                o = uniform_xover(p1, p2)
            offspring.append(o)

        for i in offspring:
            i.fitness = fitness(i.genotype)
        offspring.extend(population[:NUM_ELITISM])
        population = offspring  # Survival selection
        population.sort(key=lambda i: i.fitness, reverse=True)
        #print(population[0].fitness)
        if np.isclose(best, population[0].fitness, rtol=1/100):
            count += 1
            if count == 100:
                break
        else:
            count = 0
            best = population[0].fitness
    fitness_avg += population[0].fitness
    calls_avg += fitness.calls
    print(f"fitness calls for {n_instances} instance problem: {fitness.calls}\n with fitness: {population[0].fitness:.2%}")
    
fitness_avg /= trial
calls_avg /= trial
print(f"\n\n-----mean over {trial} try-----")
print(f"fitness calls for {n_instances} instance problem: {calls_avg:.0f}\n with fitness: {fitness_avg:.2%}")

0.546
fitness calls for 1 instance problem: 22230
 with fitness: 97.40%
0.526
fitness calls for 1 instance problem: 19230
 with fitness: 95.60%
0.527
fitness calls for 1 instance problem: 20510
 with fitness: 97.00%
0.531
fitness calls for 1 instance problem: 19070
 with fitness: 95.50%
0.526
fitness calls for 1 instance problem: 20170
 with fitness: 96.00%
0.532
fitness calls for 1 instance problem: 21910
 with fitness: 97.10%
0.528
fitness calls for 1 instance problem: 22610
 with fitness: 97.30%
0.533
fitness calls for 1 instance problem: 19950
 with fitness: 96.60%
0.528
fitness calls for 1 instance problem: 19470
 with fitness: 96.10%
0.525
fitness calls for 1 instance problem: 22190
 with fitness: 97.00%


-----mean over 10 try-----
fitness calls for 1 instance problem: 20734
 with fitness: 96.56%


Generational + Elitism + segregation (promoting diversity)  

In [8]:
POPULATION_SIZE = 120
OFFSPRING_SIZE = 0.80 #percentual
NUM_ELITISM_PERC = 0.20   # Generational + elitism


TOURNAMENT_SIZE = 4
MUTATION_PROBABILITY = .15

In [10]:
n_niches = 4
stagnation = [False] * n_niches


population = [
    Individual(genotype=[choice((1, 0)) for _ in range(NUM_LOCI)], fitness=None)
    for _ in range(POPULATION_SIZE)
]

fitness = lab9_lib.make_problem(n_instances)

for i in population:
    i.fitness = fitness(i.genotype)
populations = [population[i::n_niches] for i in range(n_niches)]  #create 5 sub-populations
for population in populations:
    population.sort(key=lambda i: i.fitness, reverse=True)
best = [populations[i][0].fitness for i in range(n_niches)]
pprint(best)
count = [0] * n_niches


for generation in range(10000):

    if np.all(stagnation):
        n_niches -= 1
        if n_niches == 0:
            break
        stagnation = [False] * n_niches
        count = [0] * n_niches
        p = [el for pop in populations for el in pop]
        populations = [p[i::n_niches] for i in range(n_niches)]  #create N-1 sub-populations
        for population in populations:
            population.sort(key=lambda i: i.fitness, reverse=True)
        best = [populations[i][0].fitness for i in range(n_niches)]

    for n in range(n_niches):
        if not stagnation[n]:
            offspring = list()
            for o in range(math.ceil(len(populations[n]) * OFFSPRING_SIZE)):
                if random() < MUTATION_PROBABILITY:
                    p = select_parent(populations[n])
                    o = mutate(p)
                else:
                    p1 = select_parent(populations[n])
                    p2 = select_parent(populations[n])
                    o = uniform_xover(p1, p2)
                offspring.append(o)

            for i in offspring:
                i.fitness = fitness(i.genotype)
            offspring.extend(populations[n][:math.floor(len(populations[n]) * NUM_ELITISM_PERC)])
            populations[n] = offspring  # Survival selection
            populations[n].sort(key=lambda i: i.fitness, reverse=True)
            print(f"({n+1}): {populations[n][0].fitness}")
            if np.isclose(best[n], populations[n][0].fitness, rtol=1/100):
                count[n] += 1
                if count[n] == 20:
                    stagnation[n] = True
            else:
                count[n] = 0
                best[n] = populations[n][0].fitness
    

print(f"fitness calls for {n_instances} instance problem: {fitness.calls}\n with fitness: {populations[0][0].fitness:.2%}")

[0.545, 0.543, 0.536, 0.542]
(1): 0.547
(2): 0.559
(3): 0.55
(4): 0.542
(1): 0.56
(2): 0.56
(3): 0.561
(4): 0.554
(1): 0.564
(2): 0.56
(3): 0.561
(4): 0.561
(1): 0.572
(2): 0.57
(3): 0.561
(4): 0.582
(1): 0.576
(2): 0.57
(3): 0.567
(4): 0.592
(1): 0.576
(2): 0.573
(3): 0.584
(4): 0.594
(1): 0.583
(2): 0.583
(3): 0.586
(4): 0.599
(1): 0.584
(2): 0.588
(3): 0.59
(4): 0.607
(1): 0.592
(2): 0.592
(3): 0.594
(4): 0.61
(1): 0.595
(2): 0.595
(3): 0.599
(4): 0.616
(1): 0.599
(2): 0.598
(3): 0.605
(4): 0.62
(1): 0.602
(2): 0.6
(3): 0.605
(4): 0.626
(1): 0.607
(2): 0.601
(3): 0.611
(4): 0.628
(1): 0.611
(2): 0.605
(3): 0.619
(4): 0.632
(1): 0.617
(2): 0.606
(3): 0.62
(4): 0.633
(1): 0.621
(2): 0.61
(3): 0.623
(4): 0.635
(1): 0.621
(2): 0.612
(3): 0.624
(4): 0.638
(1): 0.623
(2): 0.614
(3): 0.627
(4): 0.643
(1): 0.626
(2): 0.615
(3): 0.628
(4): 0.644
(1): 0.626
(2): 0.617
(3): 0.629
(4): 0.647
(1): 0.629
(2): 0.619
(3): 0.632
(4): 0.652
(1): 0.63
(2): 0.619
(3): 0.634
(4): 0.652
(1): 0.631
(2): 0

# Problem instances 2 on a 1000-loci

Generational + Elitism + segregation (promoting diversity)  

In [15]:
POPULATION_SIZE = 200
OFFSPRING_SIZE = 0.80 #percentual
NUM_ELITISM_PERC = 0.20   # Generational + elitism


TOURNAMENT_SIZE = 5
MUTATION_PROBABILITY = .15
n_instances=2

In [16]:
fitness_avg = 0
calls_avg = 0
trial = 10
for i in range(trial):
    n_niches = 4
    stagnation = [False] * n_niches
    n_stagn = 20
    perc_eq = 1

    population = [
        Individual(genotype=[choice((1, 0)) for _ in range(NUM_LOCI)], fitness=None)
        for _ in range(POPULATION_SIZE)
    ]

    fitness = lab9_lib.make_problem(n_instances)

    for i in population:
        i.fitness = fitness(i.genotype)
    populations = [population[i::n_niches] for i in range(n_niches)]  #create 5 sub-populations
    for population in populations:
        population.sort(key=lambda i: i.fitness, reverse=True)
    best = [populations[i][0].fitness for i in range(n_niches)]
    pprint(best)
    count = [0] * n_niches


    for generation in range(10000):

        if np.all(stagnation):
            n_niches -= 1
            #n_stagn *= 2
            perc_eq -= perc_eq*0.1
            if n_niches == 0:
                break
            stagnation = [False] * n_niches
            count = [0] * n_niches
            p = [el for pop in populations for el in pop]
            populations = [p[i::n_niches] for i in range(n_niches)]  #create N-1 sub-populations
            for population in populations:
                population.sort(key=lambda i: i.fitness, reverse=True)
            best = [populations[i][0].fitness for i in range(n_niches)]

        for n in range(n_niches):
            if not stagnation[n]:
                offspring = list()
                for o in range(math.ceil(len(populations[n]) * OFFSPRING_SIZE)):
                    if random() < MUTATION_PROBABILITY:
                        p = select_parent(populations[n])
                        o = mutate(p)
                    else:
                        p1 = select_parent(populations[n])
                        p2 = select_parent(populations[n])
                        o = uniform_xover(p1, p2)
                    offspring.append(o)

                for i in offspring:
                    i.fitness = fitness(i.genotype)
                offspring.extend(populations[n][:math.floor(len(populations[n]) * NUM_ELITISM_PERC)])
                populations[n] = offspring  # Survival selection
                populations[n].sort(key=lambda i: i.fitness, reverse=True)
                #print(f"({n+1}): {populations[n][0].fitness}")
                if np.isclose(best[n], populations[n][0].fitness, rtol=perc_eq/100):
                    count[n] += 1
                    if count[n] == n_stagn:
                        stagnation[n] = True
                else:
                    count[n] = 0
                    best[n] = populations[n][0].fitness
        
    fitness_avg += populations[0][0].fitness
    calls_avg += fitness.calls
    print(f"fitness calls for {n_instances} instance problem: {fitness.calls}\n with fitness: {populations[0][0].fitness:.2%}")
fitness_avg /= trial
calls_avg /= trial
print(f"\n\n-----mean over {trial} try-----")
print(f"fitness calls for {n_instances} instance problem: {calls_avg:.0f}\n with fitness: {fitness_avg:.2%}")

[0.5, 0.486, 0.5, 0.2462]
fitness calls for 2 instance problem: 85860
 with fitness: 89.80%
[0.504, 0.2595, 0.261, 0.466]
fitness calls for 2 instance problem: 75452
 with fitness: 88.80%
[0.2521, 0.496, 0.522, 0.51]
fitness calls for 2 instance problem: 82310
 with fitness: 91.60%
[0.514, 0.512, 0.24980000000000002, 0.49]
fitness calls for 2 instance problem: 97887
 with fitness: 90.40%
[0.504, 0.2491, 0.494, 0.5]
fitness calls for 2 instance problem: 40772
 with fitness: 80.60%
[0.516, 0.524, 0.2513, 0.247]
fitness calls for 2 instance problem: 71612
 with fitness: 89.40%
[0.496, 0.502, 0.528, 0.2581]
fitness calls for 2 instance problem: 88344
 with fitness: 91.40%
[0.2546, 0.532, 0.2551, 0.508]
fitness calls for 2 instance problem: 52552
 with fitness: 77.60%
[0.466, 0.478, 0.504, 0.2517]
fitness calls for 2 instance problem: 70537
 with fitness: 86.20%
[0.512, 0.494, 0.53, 0.25639999999999996]
fitness calls for 2 instance problem: 87040
 with fitness: 92.20%


-----mean over 10 tr

----------------

In [17]:
POPULATION_SIZE2 = 200 
OFFSPRING_SIZE2 = 160
NUM_ELITISM = POPULATION_SIZE2 - OFFSPRING_SIZE2   # Generational + elitism


TOURNAMENT_SIZE = 20
MUTATION_PROBABILITY = .15

In [18]:
fitness_avg = 0
calls_avg = 0
trial = 10
for i in range(trial):
    population = [
        Individual(genotype=[choice((1, 0)) for _ in range(NUM_LOCI)], fitness=None)
        for _ in range(POPULATION_SIZE2)
    ]

    fitness = lab9_lib.make_problem(n_instances)
    for i in population:
        i.fitness = fitness(i.genotype)
    population.sort(key=lambda i: i.fitness, reverse=True)
    pprint(population[0].fitness)
    best = population[0].fitness
    count = 0
    for generation in range(10000):
        offspring = list()
        for o in range(OFFSPRING_SIZE2):
            if random() < MUTATION_PROBABILITY: # self-adapt mutation probability
                # do mutation  # add more clever mutations
                p = select_parent(population)
                o = mutate(p)
            else:
                # crossover     # add more xovers
                p1 = select_parent(population)
                p2 = select_parent(population)
                o = uniform_xover(p1, p2)
            offspring.append(o)

        for i in offspring:
            i.fitness = fitness(i.genotype)
        offspring.extend(population[:NUM_ELITISM])
        population = offspring  # Survival selection
        population.sort(key=lambda i: i.fitness, reverse=True)
        #print(population[0].fitness)
        if np.isclose(best, population[0].fitness, rtol=1/100):
            count += 1
            if count == 100:
                break
        else:
            count = 0
            best = population[0].fitness
    fitness_avg += population[0].fitness
    calls_avg += fitness.calls
    print(f"fitness calls for {n_instances} instance problem: {fitness.calls}\n with fitness: {population[0].fitness:.2%}")
    
fitness_avg /= trial
calls_avg /= trial
print(f"\n\n-----mean over {trial} try-----")
print(f"fitness calls for {n_instances} instance problem: {calls_avg:.0f}\n with fitness: {fitness_avg:.2%}")

0.508
fitness calls for 2 instance problem: 60680
 with fitness: 87.80%
0.528
fitness calls for 2 instance problem: 80040
 with fitness: 88.20%
0.518
fitness calls for 2 instance problem: 71880
 with fitness: 88.00%
0.526
fitness calls for 2 instance problem: 89000
 with fitness: 90.40%
0.506
fitness calls for 2 instance problem: 70920
 with fitness: 87.80%
0.532
fitness calls for 2 instance problem: 81640
 with fitness: 89.40%
0.534
fitness calls for 2 instance problem: 91240
 with fitness: 90.80%
0.516
fitness calls for 2 instance problem: 67400
 with fitness: 90.20%
0.544
fitness calls for 2 instance problem: 58600
 with fitness: 85.60%
0.528
fitness calls for 2 instance problem: 75880
 with fitness: 90.40%


-----mean over 10 try-----
fitness calls for 2 instance problem: 74728
 with fitness: 88.86%


# Problem instances 5 on a 1000-loci

In [91]:
def mutate_n(ind: Individual,n: int) -> Individual:
    offspring = copy(ind)
    num = randint(1,n)
    for _ in range(n):
        pos = randint(0,NUM_LOCI - 1)
        offspring.genotype[pos] = 1 - offspring.genotype[pos]
    return offspring

Generational + Elitism + segregation (promoting diversity)  

In [99]:
POPULATION_SIZE = 500
OFFSPRING_SIZE = 0.80 # 80%
NUM_ELITISM_PERC = 0.20   # Generational + elitism


TOURNAMENT_SIZE = 5
MUTATION_PROBABILITY = .15
n_instances = 5

In [100]:
fitness_avg = 0
calls_avg = 0
trial = 10
for t in range(trial):
    n_niches = 5
    stagnation = [False] * n_niches
    n_stagn = 20
    perc_eq = 1

    population = [
        Individual(genotype=[choice((1, 0)) for _ in range(NUM_LOCI)], fitness=None)
        for _ in range(POPULATION_SIZE)
    ]

    fitness = lab9_lib.make_problem(n_instances)

    for i in population:
        i.fitness = fitness(i.genotype)
    populations = [population[i::n_niches] for i in range(n_niches)]  #create 5 sub-populations
    for population in populations:
        population.sort(key=lambda i: i.fitness, reverse=True)
    best = [populations[i][0].fitness for i in range(n_niches)]
    #pprint(best)
    print(t)
    count = [0] * n_niches


    for generation in range(10000):

        if np.all(stagnation):
            n_niches -= 1
            #n_stagn *= 2
            perc_eq -= perc_eq*0.1
            if n_niches == 0:
                break
            stagnation = [False] * n_niches
            count = [0] * n_niches
            p = [el for pop in populations for el in pop]
            populations = [p[i::n_niches] for i in range(n_niches)]  #create N-1 sub-populations
            for population in populations:
                population.sort(key=lambda i: i.fitness, reverse=True)
            best = [populations[i][0].fitness for i in range(n_niches)]

        for n in range(n_niches):
            if not stagnation[n]:
                offspring = list()
                for o in range(math.ceil(len(populations[n]) * OFFSPRING_SIZE)):
                    if random() < MUTATION_PROBABILITY:
                        p = select_parent(populations[n])
                        o = mutate_n(p,5)
                    else:
                        p1 = select_parent(populations[n])
                        p2 = select_parent(populations[n])
                        o = uniform_xover(p1, p2)
                    offspring.append(o)

                for i in offspring:
                    i.fitness = fitness(i.genotype)
                offspring.extend(populations[n][:math.floor(len(populations[n]) * NUM_ELITISM_PERC)])
                populations[n] = offspring  # Survival selection
                populations[n].sort(key=lambda i: i.fitness, reverse=True)
                #print(f"({n+1}): {populations[n][0].fitness}")
                if np.isclose(best[n], populations[n][0].fitness, rtol=perc_eq/100):
                    count[n] += 1
                    if count[n] == n_stagn:
                        stagnation[n] = True
                else:
                    count[n] = 0
                    best[n] = populations[n][0].fitness
        
    fitness_avg += populations[0][0].fitness
    calls_avg += fitness.calls
    print(f"fitness calls for {n_instances} instance problem: {fitness.calls}\n with fitness: {populations[0][0].fitness:.2%}")
fitness_avg /= trial
calls_avg /= trial
print(f"\n\n-----mean over {trial} try-----")
print(f"fitness calls for {n_instances} instance problem: {calls_avg:.0f}\n with fitness: {fitness_avg:.2%}")

0
fitness calls for 5 instance problem: 44920
 with fitness: 52.00%
1
fitness calls for 5 instance problem: 47000
 with fitness: 53.50%
2
fitness calls for 5 instance problem: 49460
 with fitness: 54.50%
3
fitness calls for 5 instance problem: 73240
 with fitness: 40.58%
4
fitness calls for 5 instance problem: 48480
 with fitness: 41.43%
5
fitness calls for 5 instance problem: 48400
 with fitness: 53.50%
6
fitness calls for 5 instance problem: 51960
 with fitness: 43.75%
7
fitness calls for 5 instance problem: 62760
 with fitness: 44.56%
8
fitness calls for 5 instance problem: 51080
 with fitness: 44.08%
9
fitness calls for 5 instance problem: 52494
 with fitness: 44.15%


-----mean over 10 try-----
fitness calls for 5 instance problem: 52979
 with fitness: 47.21%


------

In [105]:
POPULATION_SIZE2 = 500 
OFFSPRING_SIZE2 = 450
NUM_ELITISM = POPULATION_SIZE2 - OFFSPRING_SIZE2   # Generational + elitism


TOURNAMENT_SIZE = 5
MUTATION_PROBABILITY = .15

In [106]:
fitness_avg = 0
calls_avg = 0
trial = 20
for i in range(trial):
    population = [
        Individual(genotype=[choice((1, 0)) for _ in range(NUM_LOCI)], fitness=None)
        for _ in range(POPULATION_SIZE2)
    ]
    fitness = lab9_lib.make_problem(n_instances)
    for i in population:
        i.fitness = fitness(i.genotype)
    population.sort(key=lambda i: i.fitness, reverse=True)
    pprint(population[0].fitness)
    best = population[0].fitness
    count = 0
    for generation in range(10000):
        offspring = list()
        for o in range(OFFSPRING_SIZE2):
            if random() < MUTATION_PROBABILITY: # self-adapt mutation probability
                # do mutation  # add more clever mutations
                p = select_parent(population)
                o = mutate_n(p,1)
            else:
                # crossover     # add more xovers
                p1 = select_parent(population)
                p2 = select_parent(population)
                o = uniform_xover(p1, p2)
            offspring.append(o)

        for i in offspring:
            i.fitness = fitness(i.genotype)
        offspring.extend(population[:NUM_ELITISM])
        population = offspring  # Survival selection
        population.sort(key=lambda i: i.fitness, reverse=True)
        #print(population[0].fitness)
        if np.isclose(best, population[0].fitness, rtol=1/100):
            count += 1
            if count == 80:
                break
        else:
            count = 0
            best = population[0].fitness
    fitness_avg += population[0].fitness
    calls_avg += fitness.calls
    print(f"fitness calls for {n_instances} instance problem: {fitness.calls}\n with fitness: {population[0].fitness:.2%}")
    
fitness_avg /= trial
calls_avg /= trial
print(f"\n\n-----mean over {trial} try-----")
print(f"fitness calls for {n_instances} instance problem: {calls_avg:.0f}\n with fitness: {fitness_avg:.2%}")

0.30126
fitness calls for 5 instance problem: 70250
 with fitness: 60.50%
0.32209
fitness calls for 5 instance problem: 38300
 with fitness: 43.01%
0.3211
fitness calls for 5 instance problem: 64400
 with fitness: 46.53%
0.30717
fitness calls for 5 instance problem: 50450
 with fitness: 43.37%
0.3902
fitness calls for 5 instance problem: 70250
 with fitness: 47.88%
0.29646
fitness calls for 5 instance problem: 74300
 with fitness: 47.74%
0.28944
fitness calls for 5 instance problem: 68450
 with fitness: 48.63%
0.29802999999999996
fitness calls for 5 instance problem: 66650
 with fitness: 48.41%
0.28357
fitness calls for 5 instance problem: 60800
 with fitness: 48.35%
0.30069
fitness calls for 5 instance problem: 74750
 with fitness: 49.73%
0.3906
fitness calls for 5 instance problem: 49100
 with fitness: 55.00%
0.3127
fitness calls for 5 instance problem: 64850
 with fitness: 44.95%
0.30113999999999996
fitness calls for 5 instance problem: 68450
 with fitness: 57.00%
0.32509
fitness ca

# Problem instances 10 on a 1000-loci

Generational + Elitism + segregation (promoting diversity)  

In [141]:
POPULATION_SIZE = 1000
OFFSPRING_SIZE = 0.90 # 90%
NUM_ELITISM_PERC = 0.10   # Generational + elitism

TOURNAMENT_SIZE = 5
MUTATION_PROBABILITY = .10
n_instances = 10

In [142]:
fitness_avg = 0
calls_avg = 0
trial = 10
for t in range(trial):
    n_niches = 5
    stagnation = [False] * n_niches
    n_stagn = 20
    perc_eq = 1

    population = [
        Individual(genotype=[choice((1, 0)) for _ in range(NUM_LOCI)], fitness=None)
        for _ in range(POPULATION_SIZE)
    ]

    fitness = lab9_lib.make_problem(n_instances)

    for i in population:
        i.fitness = fitness(i.genotype)
    populations = [population[i::n_niches] for i in range(n_niches)]  #create N sub-populations
    for population in populations:
        population.sort(key=lambda i: i.fitness, reverse=True)
    best = [populations[i][0].fitness for i in range(n_niches)]
    #pprint(best)
    print(t)
    count = [0] * n_niches


    for generation in range(10000):

        if np.all(stagnation):
            n_niches -= 1
            perc_eq -= perc_eq*0.5
            if n_niches == 0:
                break
            stagnation = [False] * n_niches
            count = [0] * n_niches
            p = [el for pop in populations for el in pop]
            populations = [p[i::n_niches] for i in range(n_niches)]  #create N-1 sub-populations
            for population in populations:
                population.sort(key=lambda i: i.fitness, reverse=True)
            best = [populations[i][0].fitness for i in range(n_niches)]

        for n in range(n_niches):
            if not stagnation[n]:
                offspring = list()
                for o in range(math.ceil(len(populations[n]) * OFFSPRING_SIZE)):
                    if random() < MUTATION_PROBABILITY:
                        p = select_parent(populations[n])
                        o = mutate_n(p,10)
                    else:
                        p1 = select_parent(populations[n])
                        p2 = select_parent(populations[n])
                        o = uniform_xover(p1, p2)
                    offspring.append(o)

                for i in offspring:
                    i.fitness = fitness(i.genotype)
                offspring.extend(populations[n][:math.floor(len(populations[n]) * NUM_ELITISM_PERC)])
                populations[n] = offspring  # Survival selection
                populations[n].sort(key=lambda i: i.fitness, reverse=True)
                #print(f"({n+1}): {populations[n][0].fitness}")
                if np.isclose(best[n], populations[n][0].fitness, rtol=perc_eq/100):
                    count[n] += 1
                    if count[n] == n_stagn:
                        stagnation[n] = True
                else:
                    count[n] = 0
                    best[n] = populations[n][0].fitness
        
    fitness_avg += populations[0][0].fitness
    calls_avg += fitness.calls
    print(f"fitness calls for {n_instances} instance problem: {fitness.calls}\n with fitness: {populations[0][0].fitness:.2%}")
fitness_avg /= trial
calls_avg /= trial
print(f"\n\n-----mean over {trial} try-----")
print(f"fitness calls for {n_instances} instance problem: {calls_avg:.0f}\n with fitness: {fitness_avg:.2%}")

0
fitness calls for 10 instance problem: 99480
 with fitness: 30.64%
1
fitness calls for 10 instance problem: 182460
 with fitness: 36.43%
2
fitness calls for 10 instance problem: 113520
 with fitness: 32.99%
3
fitness calls for 10 instance problem: 111307
 with fitness: 33.00%
4
fitness calls for 10 instance problem: 138540
 with fitness: 37.27%
5
fitness calls for 10 instance problem: 108660
 with fitness: 32.49%
6
fitness calls for 10 instance problem: 109200
 with fitness: 32.99%
7
fitness calls for 10 instance problem: 123555
 with fitness: 33.59%
8
fitness calls for 10 instance problem: 110445
 with fitness: 32.99%
9
fitness calls for 10 instance problem: 106185
 with fitness: 37.25%


-----mean over 10 try-----
fitness calls for 10 instance problem: 120335
 with fitness: 33.96%


----------

In [126]:
POPULATION_SIZE2 = 4000 
OFFSPRING_SIZE2 = 3200
NUM_ELITISM = POPULATION_SIZE2 - OFFSPRING_SIZE2   # Generational + elitism


TOURNAMENT_SIZE = 100
MUTATION_PROBABILITY = .15

In [127]:
fitness_avg = 0
calls_avg = 0
trial = 10
for i in range(trial):
    population = [
        Individual(genotype=[choice((1, 0)) for _ in range(NUM_LOCI)], fitness=None)
        for _ in range(POPULATION_SIZE2)
    ]

    fitness = lab9_lib.make_problem(n_instances)
    for i in population:
        i.fitness = fitness(i.genotype)
    population.sort(key=lambda i: i.fitness, reverse=True)
    pprint(population[0].fitness)
    best = population[0].fitness
    count = 0
    for generation in range(10000):
        offspring = list()
        for o in range(OFFSPRING_SIZE2):
            if random() < MUTATION_PROBABILITY: # self-adapt mutation probability
                # do mutation  # add more clever mutations
                p = select_parent(population)
                o = mutate(p)
            else:
                # crossover     # add more xovers
                p1 = select_parent(population)
                p2 = select_parent(population)
                o = uniform_xover(p1, p2)
            offspring.append(o)

        for i in offspring:
            i.fitness = fitness(i.genotype)
        offspring.extend(population[:NUM_ELITISM])
        population = offspring  # Survival selection
        population.sort(key=lambda i: i.fitness, reverse=True)
        #print(population[0].fitness)
        if np.isclose(best, population[0].fitness, rtol=1/100):
            count += 1
            if count == 50:
                break
        else:
            count = 0
            best = population[0].fitness
    fitness_avg += population[0].fitness
    calls_avg += fitness.calls
    print(f"fitness calls for {n_instances} instance problem: {fitness.calls}\n with fitness: {population[0].fitness:.2%}")
    
fitness_avg /= trial
calls_avg /= trial
print(f"\n\n-----mean over {trial} try-----")
print(f"fitness calls for {n_instances} instance problem: {calls_avg:.0f}\n with fitness: {fitness_avg:.2%}")

0.218246833
fitness calls for 10 instance problem: 314400
 with fitness: 29.15%
0.26900409000000003
fitness calls for 10 instance problem: 199200
 with fitness: 38.64%
0.25436919
fitness calls for 10 instance problem: 192800
 with fitness: 39.48%
0.26414719000000003
fitness calls for 10 instance problem: 218400
 with fitness: 41.85%
0.25923453
fitness calls for 10 instance problem: 202400
 with fitness: 38.69%
0.26914566
fitness calls for 10 instance problem: 228000
 with fitness: 38.66%
0.24944566
fitness calls for 10 instance problem: 192800
 with fitness: 45.47%
0.351474
fitness calls for 10 instance problem: 164000
 with fitness: 35.15%
0.22190678500000002
fitness calls for 10 instance problem: 375200
 with fitness: 35.15%
0.25455688
fitness calls for 10 instance problem: 186400
 with fitness: 33.09%


-----mean over 10 try-----
fitness calls for 10 instance problem: 227360
 with fitness: 37.53%
